In [1]:
import torch
print(torch.version.cuda)

11.8


In [3]:
!pip install -U tokenizers


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch

# CUDA가 사용 가능한지 확인
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load Model and Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("../summary_service/korean_paper_summary")
tokenizer = AutoTokenizer.from_pretrained("../summary_service/korean_paper_summary")

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # 모델과 토크나이저 불러오기
# model_dir = "lcw99/t5-base-korean-text-summary"
# tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [3]:
import pandas as pd
from datasets import Dataset
import os

folder_path = "data"  # 폴더 경로
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tsv')]

# 2. 파일들을 DataFrame으로 읽어서 병합
dataframes = [pd.read_csv(file, sep='\t') for file in all_files]
merged_df = pd.concat(dataframes, ignore_index=True)


In [4]:
merged_df.isnull().sum(axis = 0)

news       0
summary    0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
merged_df = merged_df.sample(n=10000, random_state=42)

train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(test_df)

In [6]:
len(train_dataset['news'])

8000

In [7]:
def preprocess_function(examples):
    # 입력 텍스트 설정
    inputs = examples['news']
    # 입력 텍스트 토큰화 (패딩, 트렁케이션)
    model_inputs = tokenizer(
        inputs, 
        max_length=1024, 
        truncation=True, 
        padding="max_length", 
        return_tensors="pt"
    )
    
    # 요약 텍스트 토큰화 (패딩, 트렁케이션)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['summary'], 
            max_length=256, 
            truncation=True, 
            padding="max_length", 
            return_tensors="pt"
        )
    
    # 모델에 입력할 데이터에 타겟 추가
    # CUDA 디바이스로 이동
    model_inputs = {key: value.to(device) for key, value in model_inputs.items()}
    labels = labels['input_ids'].to(device)
    
    # 딕셔너리 형태로 반환
    model_inputs["labels"] = labels
    return model_inputs

# 데이터셋을 토크나이징
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_valid = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

C:\Users\euije\MakeCodes\learn\softare_engineearing\summary_service\venv\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
print(tokenized_train)
print(tokenized_train[0]['input_ids'])
print(len(tokenized_train[0]['input_ids']))
print(tokenized_train[0]['labels'])
print(len(tokenized_train[0]['labels']))

Dataset({
    features: ['news', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})
[4075, 321, 222, 10853, 222, 302, 2618, 222, 10611, 222, 3561, 222, 14, 222, 664, 222, 1736, 222, 6973, 561, 834, 222, 37895, 222, 2153, 301, 222, 4268, 333, 222, 9988, 222, 2408, 222, 4577, 279, 222, 1162, 222, 12823, 311, 222, 12081, 222, 4535, 333, 222, 3092, 322, 222, 38752, 222, 11878, 333, 222, 13170, 222, 530, 15, 222, 1749, 6054, 222, 24717, 13, 222, 3595, 16879, 222, 666, 28569, 13, 222, 2112, 27889, 222, 369, 1299, 222, 5542, 1036, 274, 222, 25, 349, 222, 2258, 389, 222, 1592, 222, 388, 31664, 222, 353, 222, 2402, 477, 1219, 222, 1588, 36949, 222, 2248, 305, 222, 160, 224, 248, 1219, 302, 222, 44, 53, 7, 40, 222, 2017, 222, 1736, 222, 6973, 222, 710, 222, 4268, 302, 222, 403, 5471, 779, 222, 4681, 222, 8983, 160, 224, 249, 222, 4577, 381, 222, 1046, 322, 222, 2258, 222, 2402, 3482, 421, 222, 3653, 302, 222, 9238, 459, 333, 222, 1808, 701, 293, 222, 

In [9]:
from transformers import Trainer, TrainingArguments

# 훈련 설정
training_args = TrainingArguments(
    output_dir="./checkpoint",
    evaluation_strategy="epoch",  # 매 epoch마다 평가
    save_strategy="epoch",        # 매 epoch마다 체크포인트 저장
    num_train_epochs=30,  # 최대 30 epoch
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=3,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,  # 훈련 종료 후 가장 좋은 모델을 불러오기
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer
)


C:\Users\euije\MakeCodes\learn\softare_engineearing\summary_service\venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\euije\AppData\Local\Temp\ipykernel_41948\199574312.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:

# 파인튜닝 시작
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.261200,0.247479
2,0.236900,0.242154


KeyboardInterrupt: 

In [17]:
# 평가
trainer.evaluate()

KeyboardInterrupt: 

In [11]:
model.save_pretrained("./korean_summary_finetuned")
tokenizer.save_pretrained("./korean_summary_finetuned")

('./korean_summary_finetuned\\tokenizer_config.json',
 './korean_summary_finetuned\\special_tokens_map.json',
 './korean_summary_finetuned\\tokenizer.json')

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
f1_model = AutoModelForSeq2SeqLM.from_pretrained("../summary_service/korean_paper_summary")
f1_tokenizer = AutoTokenizer.from_pretrained("../summary_service/korean_paper_summary")

In [7]:
from transformers import AutoTokenizer, BartForConditionalGeneration
f2_model = AutoModelForSeq2SeqLM.from_pretrained("./korean_summary_finetuned")
f2_tokenizer = AutoTokenizer.from_pretrained("./korean_summary_finetuned")

In [8]:
import pandas as pd
from datasets import Dataset
import os

folder_path = "diff"  # 폴더 경로
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tsv')]

# 2. 파일들을 DataFrame으로 읽어서 병합
dataframes = [pd.read_csv(file, sep='\t') for file in all_files]
merged_df = pd.concat(dataframes, ignore_index=True)


In [12]:

for index, row in merged_df.iterrows():
    print(f1(row[0]))
    print(f2(row[0]))
    print()

C:\Users\euije\AppData\Local\Temp\ipykernel_7644\1196624857.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f1(row[0]))


중국인 한국어 학습자의 한국어 이중모음 산출 특성이 한국인과 다를 것이라는 가설 하에 한국인과 중국인의 한국어 이중모음의 발화 양상을 음성 분석하여 비교한 결과, 두 집단은 차이가 없는 것으로 나타났고, 활음 구간을 시작지점과 종결지점으로 2등분으로 나누어 포르만트를 분석한 결과, 두 집단은 이중모음 /ᅭ/와 /ᅲ/의 정확한 발화에 어려움을 가지고 있음을 알 수 있다.


C:\Users\euije\AppData\Local\Temp\ipykernel_7644\1196624857.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f2(row[0]))


이 논문은 중국인 한국어 학습자의 한국어 이중모음 산출 특성이 한국인과 다를 것이라는 가설 하에 한국인과 중국인의 한국어 이중모음의 발화 양상을 음성 분석해 비교했다. 두 집단의 이중모음 산출 시 나타나는 음향학적 특성을 비교해 어떠한 차이를 가지는지 살펴보았으며 또한 그 차이가 통계적으로 유의미한가에 관해 알아보았다.

서비스 부문은 제조업과 다른 특성을 지니므로 전통적인 6시그마 전개방식을 서비스 분야에 적용하는 과정에 여러 가지 어려움이 제기되고 있다. 이에 본 연구에서는 서비스 부분의 프로세스 및 상품개발과정에 효과적으로 적용할 수 있는 DFSS/S 체계의 재정립 방안을 논의하고 그 적용 사례를 소개하고자 한다.
본고에서는 서비스 부분의 프로세스 및 상품개발과정에 효과적으로 적용할 수 있는 DFSS/S 체계의 재정립 방안을 논의하고 그 적용 사례를 소개하고자 한다. DFSS/S 로드맵의 조정·보완, 고객 분석 및 CTQ(Y) 도출, 컨셉 개발을 위한 아이디어 도출 및 평가, 그리고 Y-y-X 연계도를 통한 효율적인 프로세스 설계방안 등이 기술되며, 이러한 개념적 구조를 서비스 상품의 개발에 적용한 기업 사례를 다루고 있다.

안전행정부는 전국의 재난상황을 24시간 모니터링하면서 사고 발생 시 위기관리체계를 신속히 가동해 대처하도록 하는 법적 임무를 맡고 있다. 하지만 강 장관은 대통령에게 지연 보고를 했다는 점, 정부의 초기 대응을 갈팡질팡하게 만든 단초를 제공했다는 점만으로도 강 장관의 책임은 한없이 무겁다. 더 놀라운 것은 강 장관이 자신의 권한과 책임의 범위를 혼동하고 있다는 점이다. 하지만 중대본은 대규모 재난의 예방·대비·대응·복구에 관한 사항을 총괄 조정하는 기구다. 24시간 정확한 상황을 파악해 전파하고 부처별 역할을 분담·조정해 합동 지원책을 마련해야 할 책임이 있다.
세월호 참사의 책임소재를 놓고 따지자면 선장·선원과 선사(船社), 그리고 해경을 우선 꼽아야겠지만 안전행정부는 전국의 재난상황을 24시간 모니터링하면서 사고 발생 시 위기관

KeyboardInterrupt: 

In [10]:

def f1(report):
    inputs1 = f1_tokenizer(report, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = f1_model.generate(inputs1["input_ids"], num_beams=4, max_length=350)
    return f1_tokenizer.decode(summary_ids[0], skip_special_tokens=True)



In [11]:
def f2(report):
    inputs2 = f2_tokenizer(report, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = f2_model.generate(inputs2["input_ids"], num_beams=4, max_length=350)
    return f2_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    



In [15]:
report = "안전행정부는 전국의 재난상황을 24시간 모니터링하면서 사고 발생 시 위기관리체계를 신속히 가동해 대처하도록 하는 법적 임무를 맡고 있다. 하지만 강 장관은 대통령에게 지연 보고를 했다는 점, 정부의 초기 대응을 갈팡질팡하게 만든 단초를 제공했다는 점만으로도 강 장관의 책임은 한없이 무겁다. 더 놀라운 것은 강 장관이 자신의 권한과 책임의 범위를 혼동하고 있다는 점이다. 하지만 중대본은 대규모 재난의 예방·대비·대응·복구에 관한 사항을 총괄 조정하는 기구다. 24시간 정확한 상황을 파악해 전파하고 부처별 역할을 분담·조정해 합동 지원책을 마련해야 할 책임이 있다."

inputs = tokenizer(report, return_tensors="pt", max_length=1024, truncation=True).to(device)
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=350)
result = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(result)

안전행정부는 전국의 재난상황을 24시간 모니터링하면서 사고 발생 시 위기관리체계를 신속히 가동해 대처하도록 하는 법적 임무를 맡고 있다.
